# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 2.0.10 which is incompatible.


The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726671e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
# env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
# states = env_info.vector_observations                  # get the current state (for each agent)
# scores = np.zeros(num_agents)                          # initialize the score (for each agent)
# while True:
#     actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#     actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#     env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#     next_states = env_info.vector_observations         # get next state (for each agent)
#     rewards = env_info.rewards                         # get reward (for each agent)
#     dones = env_info.local_done                        # see if episode finished
#     scores += env_info.rewards                         # update the score (for each agent)
#     states = next_states                               # roll over states to next time step
#     if np.any(dones):                                  # exit loop if episode finished
#         break
# print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [6]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [7]:
from  collections  import deque
import torch
import time
from ddpg_agent import Agent
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
env_info = env.reset(train_mode=True)[brain_name]
agent = Agent(state_size=state_size, action_size=action_size, random_seed=8)

In [9]:
def ddpg(agent, n_episodes=30000, max_t=2000, print_every=50, print_episode=False):
    scores_deque = deque(maxlen=100)                           # Window of maximum length 100
    scores_global = []
        
    time_start = time.time()
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
        states = env_info.vector_observations                  # get the current state for each agent
        scores_agent = np.zeros(num_agents)                    # initialize the score for each agent
        agent.reset()
        average_score = 0

        for timestep in range(max_t):
            actions = agent.act(states)
            env_info = env.step(actions)[brain_name]           # send all actions to the environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            agent.step(states, actions, rewards, next_states, dones, timestep)
            states = next_states                               # roll over states to next time step
            scores_agent += rewards                            # update the score for each agent    
            if np.any(dones):                                  # exit loop if episode finished
                break
        
        score = np.mean(scores_agent)
        scores_deque.append(score)
        average_score = np.mean(scores_deque)
        scores_global.append(score)

        time_end = time.time()
        
        if print_episode:
            print('Episode: {}, Average Score: {:.2f}, Max: {:.2f}, Min: {:.2f} '\
                .format(i_episode, score, np.max(scores_agent), np.min(scores_agent)))

        if i_episode % print_every == 0:            
            time_elapsed = (int)(time_end - time_start) 
            print('Episode {}\t Average Score: {:.2f}\t Time: {:02}:{:02}:{:02}'\
                  .format(i_episode, np.mean(scores_deque), time_elapsed//3600, time_elapsed%3600//60, time_elapsed%60))  
            
        if len(scores_deque) == 100 and np.mean(scores_deque) >= 30:
            torch.save(agent.actor_local.state_dict(), 'weights/checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'weights/checkpoint_critic.pth')  
            print('\nEnvironment solved in {:d} episodes!'.format(i_episode))
            break
            
    return scores_global

In [12]:
import workspace_utils
from workspace_utils import active_session

In [13]:
with active_session():
    scores = ddpg(agent)

Episode 50	 Average Score: 1.50	 Time: 00:07:38
Episode 100	 Average Score: 2.30	 Time: 00:15:34
Episode 150	 Average Score: 2.94	 Time: 00:23:56
Episode 200	 Average Score: 3.02	 Time: 00:32:53
Episode 250	 Average Score: 3.28	 Time: 00:42:28
Episode 300	 Average Score: 3.48	 Time: 00:52:42
Episode 350	 Average Score: 3.70	 Time: 01:03:35
Episode 400	 Average Score: 3.76	 Time: 01:15:10
Episode 450	 Average Score: 3.45	 Time: 01:27:45
Episode 500	 Average Score: 3.97	 Time: 01:41:13
Episode 550	 Average Score: 4.69	 Time: 01:55:36
Episode 600	 Average Score: 4.18	 Time: 02:11:03
Episode 650	 Average Score: 3.51	 Time: 02:27:40
Episode 700	 Average Score: 3.72	 Time: 02:45:29
Episode 750	 Average Score: 4.00	 Time: 03:04:37


KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[10, 5])
plt.rcParams.update({'font.size': 14})
ax.plot(scores, "-", c="blue", alpha=0.5)
ax.set_xlabel("Episode #")
ax.set_ylabel("Score")
ax.grid(which="major")
ax.axhline(30, c="green", linewidth=3, alpha=0.5)
ax.legend(["Reward Score per Episode", "Target Score"])
fig.tight_layout()
fig.savefig("result.png")

In [ ]:
env.close()